# SQL Functions, based `OllamaFunctions`

## Setup

### Imports

In [1]:
import os
import pickle

import db_connect
from test_data import TestData

from langsmith import Client

from langchain_core.language_models import BaseLanguageModel

from langchain.prompts import PromptTemplate

from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits.sql import base
from langchain.agents.agent import AgentExecutor
from langchain.agents import tool

from langchain_experimental.llms.ollama_functions import OllamaFunctions, DEFAULT_RESPONSE_FUNCTION, DEFAULT_SYSTEM_TEMPLATE
from langchain_experimental.llms.ollama_functions import convert_to_ollama_tool, parse_response
from langchain_core.utils.function_calling import convert_to_openai_function

from langchain_core.prompts import PromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.agents.output_parsers.openai_functions import OpenAIFunctionsAgentOutputParser

from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_core.messages.system import SystemMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.messages.ai import AIMessage

from langchain.schema.agent import AgentFinish

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate

### LangSmith

In [2]:
os.environ["LANGCHAIN_PROJECT"] = "text2sql"
client = Client()

### Load models

In [3]:
llama3 = OllamaFunctions(model="llama3:8b", format="json")
llama3_inst = OllamaFunctions(model="llama3:instruct", format="json")
llama3_inst_q8 = OllamaFunctions(model="llama3:8b-instruct-q8_0", format="json")
llama3_inst_fp16 = OllamaFunctions(model="llama3:8b-instruct-fp16", format="json")

llama3_text = OllamaFunctions(model="llama3:8b-text", format="json")

### Connect to DB with Readonly role

In [4]:
db = db_connect.get_db()

#### Check connection

In [5]:
db.run("select * from passenger")

"[(1, 'John'), (2, 'James'), (3, 'Poul'), (4, 'Christofer'), (5, 'Superman'), (6, 'Donald'), (7, 'Douglas'), (8, 'Dwight'), (9, 'Earl'), (10, 'Edgar'), (11, 'Edmund'), (12, 'Edwin'), (13, 'Elliot'), (14, 'Eric'), (15, 'Ernest'), (16, 'Ethan'), (17, 'Ezekiel'), (18, 'Felix'), (19, 'Franklin'), (20, 'Frederick'), (21, 'Gabriel'), (22, 'Joseph'), (23, 'Joshua'), (24, 'Julian'), (25, 'Alice'), (26, 'Bob'), (27, 'Charlie'), (28, 'David'), (29, 'Emily'), (30, 'Frank'), (31, 'George'), (32, 'Helen'), (33, 'Irene'), (34, 'Jack'), (35, 'Kate'), (36, 'Leo'), (37, 'Mary'), (38, 'Nancy'), (39, 'Oliver'), (40, 'Paul'), (41, 'Qiana'), (42, 'Robert'), (43, 'Samantha'), (44, 'Thomas'), (45, 'Victoria')]"

-----

## Create agent

In [6]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

func = convert_to_openai_function(WeatherSearch)
print(func)

{'name': 'WeatherSearch', 'description': 'Call this with an airport code to get the weather at that airport', 'parameters': {'type': 'object', 'properties': {'airport_code': {'description': 'airport code to get weather for', 'type': 'string'}}, 'required': ['airport_code']}}


In [7]:
model_with_funcs = llama3_inst_q8.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        },
        DEFAULT_RESPONSE_FUNCTION
    ],
)

In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "# Personality\nYou are friendly ai assistent.\n\n# Todo\nAnswer the user. If you need, you can use some of this tools."),
    ("user", "{question}")
])

In [9]:
model_with_prompt = prompt | model_with_funcs 

In [10]:
def get_current_weather(**kwargs):
    return f"451 degrees in {kwargs['unit']} at {kwargs['location']}"

In [11]:
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = { 
            "get_current_weather": get_current_weather,
        }
        return tools[result.tool](**result.tool_input)

In [12]:
model = model_with_funcs | OpenAIFunctionsAgentOutputParser() | route

In [13]:
res = model.batch(["Hi!", "What the weather in New-York?"])

In [14]:
print(res)

["Hello! It's nice to chat with you. What would you like to talk about?", '451 degrees in celsius at New York, NY']
